In [55]:
regional_key = 'Landeshauptstadt München'

import sys
sys.path.append('/Users/felix/Documents/thesis')

import pharmada.regkey as rk
import pharmada.geometry as geo
import pharmada.pharmacies as ph
import pharmada.overpass as op
import osm2geojson as o2g
import geopandas as gpd
import folium

In [56]:
RegKey = rk.RegKey(regional_key)
AreaGeometry = geo.AreaGeometry(RegKey)

In [57]:
query = fr"""
[out:json];
    
//set up area given by regional_key as search boundary
(
relation({AreaGeometry.osm_id});
map_to_area;
) ->.boundary;

wr(area.boundary)[type=boundary][boundary=postal_code]->.results;

// output full geometry of the results set     
.results out geom qt;
"""

response = op.query_overpass(query)

In [58]:
result = o2g.json2geojson(response)
# Convert the geometry to a GeoDataFrame
geometry = gpd.GeoDataFrame.from_features(result, crs='EPSG:4326')
geometry['postal_code'] = geometry['tags'].apply(lambda tags: tags.get('postal_code'))


In [59]:
style_kwds = {
    'fillOpacity': 0.8,
}
map = AreaGeometry.geometry.explore()
geometry.explore(m=map, cmap='tab20', column='postal_code', style_kwds=style_kwds)

In [70]:
geom = AreaGeometry.geometry.to_crs(AreaGeometry.geometry.estimate_utm_crs())
centroid = geom.centroid.to_crs(geom.crs)


m = folium.Map(location=[centroid.y[0], centroid.x[0]], zoom_start=18)
folium.TileLayer('openstreetmap').add_to(m)

In [71]:
m